In [1]:
import pandas_datareader.data as web
 
import pandas as pd
 
import numpy as np
 
import datetime as dt
 
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from config import user, password
 


In [2]:
# SET START AND END DATES OF THE SERIES
sdt = dt.datetime(1995, 1, 1)
 
edt = dt.datetime(2019, 11, 1)
 


In [4]:
cpi = web.DataReader("CPIAUCNS", "fred", sdt, edt)
 
cpi.head()



,CPIAUCNS
DATE,
1995-01-01,150.3
1995-02-01,150.9
1995-03-01,151.4
1995-04-01,151.9
1995-05-01,152.2


In [6]:
aaa = web.DataReader("AAA", "fred", sdt, edt)
 
aaa.head()



,AAA
DATE,
1995-01-01,8.46
1995-02-01,8.26
1995-03-01,8.12
1995-04-01,8.03
1995-05-01,7.65


In [7]:
df = pd.merge(cpi,aaa, on="DATE", how ="left")
df

,CPIAUCNS,AAA
DATE,,
1995-01-01,150.300,8.46
1995-02-01,150.900,8.26
1995-03-01,151.400,8.12
1995-04-01,151.900,8.03
1995-05-01,152.200,7.65
...,...,...
2019-06-01,256.143,3.42
2019-07-01,256.571,3.29
2019-08-01,256.558,2.98


In [8]:
baa = web.DataReader("BAA", "fred", sdt, edt)
 
baa.head()

,BAA
DATE,
1995-01-01,9.08
1995-02-01,8.85
1995-03-01,8.70
1995-04-01,8.60
1995-05-01,8.20


In [9]:
aaa_baa = pd.merge(aaa,baa, on="DATE", how ="left")
aaa_baa



,AAA,BAA
DATE,,
1995-01-01,8.46,9.08
1995-02-01,8.26,8.85
1995-03-01,8.12,8.70
1995-04-01,8.03,8.60
1995-05-01,7.65,8.20
...,...,...
2019-06-01,3.42,4.46
2019-07-01,3.29,4.28
2019-08-01,2.98,3.87


In [10]:
T10Y2YM = web.DataReader("T10Y2YM", "fred", sdt, edt)
 
T10Y2YM.head()



,T10Y2YM
DATE,
1995-01-01,0.27
1995-02-01,0.36
1995-03-01,0.42
1995-04-01,0.49
1995-05-01,0.46


In [11]:
DTB3 = web.DataReader("DTB3", "fred", sdt, edt)
 
DTB3.head()
DTB3_C = DTB3.dropna()

In [13]:

DTB3_M = DTB3_C.resample('1M').mean()



In [14]:
SPY = web.get_data_yahoo('^GSPC', start=dt.datetime(1994, 1, 1), end=dt.datetime(2019, 12, 1))


SPY = SPY.drop(["High", "Low", "Open","Volume", "Adj Close"], axis=1)
SPY = SPY.resample('1M').mean()

SPY_Pct_Chg = SPY.pct_change(12)
SPY_Pct_Chg.head(20)

SPY_Merged = pd.merge(SPY,SPY_Pct_Chg, on="Date", how ="left")

In [15]:
SPY_Merged

,Close_x,Close_y
Date,,
1994-01-31,472.992857,NaN
1994-02-28,471.579999,NaN
1994-03-31,463.807827,NaN
1994-04-30,447.232107,NaN
1994-05-31,450.898573,NaN
...,...,...
2019-07-31,2996.113636,0.072475
2019-08-31,2897.450451,0.013867
2019-09-30,2982.156006,0.027798


In [16]:
SPY_Merged.rename(columns = {'Close_x':'Closing Price', 'Close_y':'12mFR'
                             }, inplace = True) 

In [17]:
SPY_Merged

,Closing Price,12mFR
Date,,
1994-01-31,472.992857,NaN
1994-02-28,471.579999,NaN
1994-03-31,463.807827,NaN
1994-04-30,447.232107,NaN
1994-05-31,450.898573,NaN
...,...,...
2019-07-31,2996.113636,0.072475
2019-08-31,2897.450451,0.013867
2019-09-30,2982.156006,0.027798


In [18]:
SPY_Merged_C = SPY_Merged.dropna()
SPY_Merged_C

,Closing Price,12mFR
Date,,
1995-01-31,465.250000,-0.016370
1995-02-28,481.924737,0.021936
1995-03-31,493.153047,0.063270
1995-04-30,507.905791,0.135665
1995-05-31,523.813640,0.161711
...,...,...
2019-07-31,2996.113636,0.072475
2019-08-31,2897.450451,0.013867
2019-09-30,2982.156006,0.027798


#Inserting Data

In [19]:
rds_connection_string = user + ":" + password + "@localhost:5432/ETLStock"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [1]:
engine.table_names()

NameError: name 'engine' is not defined

In [21]:
aaa_baa.to_sql(name='Bonds', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_Bonds"
DETAIL:  Key ("DATE")=(1995-01-01) already exists.
 [SQL: 'INSERT INTO "Bonds" ("DATE", "AAA", "BAA") VALUES (%(DATE)s, %(AAA)s, %(BAA)s)'] [parameters: ({'DATE': datetime.datetime(1995, 1, 1, 0, 0), 'AAA': 8.46, 'BAA': 9.08}, {'DATE': datetime.datetime(1995, 2, 1, 0, 0), 'AAA': 8.26, 'BAA': 8.85}, {'DATE': datetime.datetime(1995, 3, 1, 0, 0), 'AAA': 8.12, 'BAA': 8.7}, {'DATE': datetime.datetime(1995, 4, 1, 0, 0), 'AAA': 8.03, 'BAA': 8.6}, {'DATE': datetime.datetime(1995, 5, 1, 0, 0), 'AAA': 7.65, 'BAA': 8.2}, {'DATE': datetime.datetime(1995, 6, 1, 0, 0), 'AAA': 7.3, 'BAA': 7.9}, {'DATE': datetime.datetime(1995, 7, 1, 0, 0), 'AAA': 7.41, 'BAA': 8.04}, {'DATE': datetime.datetime(1995, 8, 1, 0, 0), 'AAA': 7.57, 'BAA': 8.19}  ... displaying 10 of 298 total bound parameter sets ...  {'DATE': datetime.datetime(2019, 9, 1, 0, 0), 'AAA': 3.03, 'BAA': 3.91}, {'DATE': datetime.datetime(20

In [22]:
cpi.to_sql(name='CPI', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "CPIAUCNS" of relation "CPI" does not exist
LINE 1: INSERT INTO "CPI" ("DATE", "CPIAUCNS") VALUES ('1995-01-01T0...
                                   ^
 [SQL: 'INSERT INTO "CPI" ("DATE", "CPIAUCNS") VALUES (%(DATE)s, %(CPIAUCNS)s)'] [parameters: ({'DATE': datetime.datetime(1995, 1, 1, 0, 0), 'CPIAUCNS': 150.3}, {'DATE': datetime.datetime(1995, 2, 1, 0, 0), 'CPIAUCNS': 150.9}, {'DATE': datetime.datetime(1995, 3, 1, 0, 0), 'CPIAUCNS': 151.4}, {'DATE': datetime.datetime(1995, 4, 1, 0, 0), 'CPIAUCNS': 151.9}, {'DATE': datetime.datetime(1995, 5, 1, 0, 0), 'CPIAUCNS': 152.2}, {'DATE': datetime.datetime(1995, 6, 1, 0, 0), 'CPIAUCNS': 152.5}, {'DATE': datetime.datetime(1995, 7, 1, 0, 0), 'CPIAUCNS': 152.5}, {'DATE': datetime.datetime(1995, 8, 1, 0, 0), 'CPIAUCNS': 152.9}  ... displaying 10 of 298 total bound parameter sets ...  {'DATE': datetime.datetime(2019, 9, 1, 0, 0), 'CPIAUCNS': 256.759}, {'DATE': datetime.datetime(2019, 10, 1, 